In [1]:
import pandas as pd
import statistics as stats

In [2]:
complete_human_annotation = pd.read_csv('../data/gencode.v45.annotation.gff3', sep='\t', header=None, comment='#')

In [3]:
complete_human_annotation

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000290825.1;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...
...,...,...,...,...,...,...,...,...,...
3425351,chrM,ENSEMBL,transcript,15888,15953,.,+,.,ID=ENST00000387460.2;Parent=ENSG00000210195.2;...
3425352,chrM,ENSEMBL,exon,15888,15953,.,+,.,ID=exon:ENST00000387460.2:1;Parent=ENST0000038...
3425353,chrM,ENSEMBL,gene,15956,16023,.,-,.,ID=ENSG00000210196.2;gene_id=ENSG00000210196.2...
3425354,chrM,ENSEMBL,transcript,15956,16023,.,-,.,ID=ENST00000387461.2;Parent=ENSG00000210196.2;...


In [4]:
# keeping usefull line
feature_human_annotation = complete_human_annotation[complete_human_annotation[2].isin(['gene', 'transcript', 'CDS'])]

In [5]:
# keeping usefull column
human_annotation = feature_human_annotation.loc[:,[0,2,3,4,8]]

In [6]:
human_annotation

,0,2,3,4,8
0,chr1,gene,11869,14409,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...
1,chr1,transcript,11869,14409,ID=ENST00000456328.2;Parent=ENSG00000290825.1;...
5,chr1,gene,12010,13670,ID=ENSG00000223972.6;gene_id=ENSG00000223972.6...
6,chr1,transcript,12010,13670,ID=ENST00000450305.2;Parent=ENSG00000223972.6;...
13,chr1,gene,14696,24886,ID=ENSG00000227232.6;gene_id=ENSG00000227232.6...
...,...,...,...,...,...
3425348,chrM,CDS,14747,15887,ID=CDS:ENST00000361789.2;Parent=ENST0000036178...
3425350,chrM,gene,15888,15953,ID=ENSG00000210195.2;gene_id=ENSG00000210195.2...
3425351,chrM,transcript,15888,15953,ID=ENST00000387460.2;Parent=ENSG00000210195.2;...
3425353,chrM,gene,15956,16023,ID=ENSG00000210196.2;gene_id=ENSG00000210196.2...


In [7]:
gene = human_annotation[human_annotation[2].isin(['gene'])]
gene

,0,2,3,4,8
0,chr1,gene,11869,14409,ID=ENSG00000290825.1;gene_id=ENSG00000290825.1...
5,chr1,gene,12010,13670,ID=ENSG00000223972.6;gene_id=ENSG00000223972.6...
13,chr1,gene,14696,24886,ID=ENSG00000227232.6;gene_id=ENSG00000227232.6...
25,chr1,gene,17369,17436,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...
28,chr1,gene,29554,31109,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...
...,...,...,...,...,...
3425337,chrM,gene,14149,14673,ID=ENSG00000198695.2;gene_id=ENSG00000198695.2...
3425342,chrM,gene,14674,14742,ID=ENSG00000210194.1;gene_id=ENSG00000210194.1...
3425345,chrM,gene,14747,15887,ID=ENSG00000198727.2;gene_id=ENSG00000198727.2...
3425350,chrM,gene,15888,15953,ID=ENSG00000210195.2;gene_id=ENSG00000210195.2...


In [8]:
only_protein_coding = gene[8].apply(lambda x : "gene_type=protein_coding" in x)
gene = gene[only_protein_coding]
with_readthrough = gene[8].apply(lambda x : "readthrough" in x)
gene = gene[~with_readthrough]
gene['ID'] = gene[8].apply(lambda x : x.split(';')[0].lstrip('ID='))
coding_gene = set(gene['ID'].tolist())

In [9]:
len(coding_gene)

19395

In [10]:
def infogff2dict(x):
    return dict( (k,v) for k,v in [element.split('=') for element in x.split(';')])

In [11]:
transcript = human_annotation[human_annotation[2].isin(['transcript'])]
transcript

,0,2,3,4,8
1,chr1,transcript,11869,14409,ID=ENST00000456328.2;Parent=ENSG00000290825.1;...
6,chr1,transcript,12010,13670,ID=ENST00000450305.2;Parent=ENSG00000223972.6;...
14,chr1,transcript,14696,24886,ID=ENST00000488147.2;Parent=ENSG00000227232.6;...
26,chr1,transcript,17369,17436,ID=ENST00000619216.1;Parent=ENSG00000278267.1;...
29,chr1,transcript,29554,31097,ID=ENST00000473358.1;Parent=ENSG00000243485.5;...
...,...,...,...,...,...
3425338,chrM,transcript,14149,14673,ID=ENST00000361681.2;Parent=ENSG00000198695.2;...
3425343,chrM,transcript,14674,14742,ID=ENST00000387459.1;Parent=ENSG00000210194.1;...
3425346,chrM,transcript,14747,15887,ID=ENST00000361789.2;Parent=ENSG00000198727.2;...
3425351,chrM,transcript,15888,15953,ID=ENST00000387460.2;Parent=ENSG00000210195.2;...


In [12]:
transcript_from_coding_gene = transcript[8].apply(lambda x : infogff2dict(x)['gene_id'] in coding_gene)

In [13]:
transcript = transcript[transcript_from_coding_gene]
transcript

,0,2,3,4,8
59,chr1,transcript,65419,71585,ID=ENST00000641515.2;Parent=ENSG00000186092.7;...
315,chr1,transcript,450740,451678,ID=ENST00000426406.4;Parent=ENSG00000284733.2;...
442,chr1,transcript,685716,686654,ID=ENST00000332831.5;Parent=ENSG00000284662.2;...
977,chr1,transcript,923923,944574,ID=ENST00000616016.5;Parent=ENSG00000187634.13...
1010,chr1,transcript,923923,944574,ID=ENST00000618323.5;Parent=ENSG00000187634.13...
...,...,...,...,...,...
3425313,chrM,transcript,10470,10766,ID=ENST00000361335.1;Parent=ENSG00000212907.2;...
3425319,chrM,transcript,10760,12137,ID=ENST00000361381.2;Parent=ENSG00000198886.2;...
3425333,chrM,transcript,12337,14148,ID=ENST00000361567.2;Parent=ENSG00000198786.2;...
3425338,chrM,transcript,14149,14673,ID=ENST00000361681.2;Parent=ENSG00000198695.2;...


In [14]:
only_protein_coding = transcript[8].apply(lambda x : infogff2dict(x)['transcript_type'] == 'protein_coding')
transcript = transcript[only_protein_coding]
transcript['ID'] = transcript[8].apply(lambda x : x.split(';')[0].lstrip('ID='))
coding_transcript = set(transcript['ID'].tolist())
transcript

/tmp/ipykernel_8902/3773129726.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript['ID'] = transcript[8].apply(lambda x : x.split(';')[0].lstrip('ID='))


,0,2,3,4,8,ID
59,chr1,transcript,65419,71585,ID=ENST00000641515.2;Parent=ENSG00000186092.7;...,ENST00000641515.2
315,chr1,transcript,450740,451678,ID=ENST00000426406.4;Parent=ENSG00000284733.2;...,ENST00000426406.4
442,chr1,transcript,685716,686654,ID=ENST00000332831.5;Parent=ENSG00000284662.2;...,ENST00000332831.5
977,chr1,transcript,923923,944574,ID=ENST00000616016.5;Parent=ENSG00000187634.13...,ENST00000616016.5
1010,chr1,transcript,923923,944574,ID=ENST00000618323.5;Parent=ENSG00000187634.13...,ENST00000618323.5
...,...,...,...,...,...,...
3425313,chrM,transcript,10470,10766,ID=ENST00000361335.1;Parent=ENSG00000212907.2;...,ENST00000361335.1
3425319,chrM,transcript,10760,12137,ID=ENST00000361381.2;Parent=ENSG00000198886.2;...,ENST00000361381.2
3425333,chrM,transcript,12337,14148,ID=ENST00000361567.2;Parent=ENSG00000198786.2;...,ENST00000361567.2
3425338,chrM,transcript,14149,14673,ID=ENST00000361681.2;Parent=ENSG00000198695.2;...,ENST00000361681.2


In [15]:
cds = human_annotation[human_annotation[2].isin(['CDS'])]
cds

,0,2,3,4,8
62,chr1,CDS,65565,65573,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
65,chr1,CDS,69037,70008,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
317,chr1,CDS,450740,451678,ID=CDS:ENST00000426406.4;Parent=ENST0000042640...
444,chr1,CDS,685716,686654,ID=CDS:ENST00000332831.5;Parent=ENST0000033283...
979,chr1,CDS,924432,924948,ID=CDS:ENST00000616016.5;Parent=ENST0000061601...
...,...,...,...,...,...
3425315,chrM,CDS,10470,10766,ID=CDS:ENST00000361335.1;Parent=ENST0000036133...
3425321,chrM,CDS,10760,12137,ID=CDS:ENST00000361381.2;Parent=ENST0000036138...
3425335,chrM,CDS,12337,14148,ID=CDS:ENST00000361567.2;Parent=ENST0000036156...
3425340,chrM,CDS,14149,14673,ID=CDS:ENST00000361681.2;Parent=ENST0000036168...


In [16]:
cds_from_coding_transcript = cds[8].apply(lambda x : infogff2dict(x)['transcript_id'] in coding_transcript)
cds = cds[cds_from_coding_transcript]
cds

,0,2,3,4,8
62,chr1,CDS,65565,65573,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
65,chr1,CDS,69037,70008,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
317,chr1,CDS,450740,451678,ID=CDS:ENST00000426406.4;Parent=ENST0000042640...
444,chr1,CDS,685716,686654,ID=CDS:ENST00000332831.5;Parent=ENST0000033283...
979,chr1,CDS,924432,924948,ID=CDS:ENST00000616016.5;Parent=ENST0000061601...
...,...,...,...,...,...
3425315,chrM,CDS,10470,10766,ID=CDS:ENST00000361335.1;Parent=ENST0000036133...
3425321,chrM,CDS,10760,12137,ID=CDS:ENST00000361381.2;Parent=ENST0000036138...
3425335,chrM,CDS,12337,14148,ID=CDS:ENST00000361567.2;Parent=ENST0000036156...
3425340,chrM,CDS,14149,14673,ID=CDS:ENST00000361681.2;Parent=ENST0000036168...


In [17]:
repartition = dict()  # repartion[gene][transcript]
for index, row in cds.iterrows():
    row=list(row)
    chr = row[0]
    start = row[2]
    end = row[3]
    info = dict( (k,v) for k,v in [element.split('=') for element in row[4].split(';')])
    gene_id = info['gene_id']
    transcript_id = info['transcript_id']
    # if info['transcript_type'] not in ['protein_coding', 'protein_coding_LoF']:
    #     if info['transcript_type'] not in ['nonsense_mediated_decay', 'non_stop_decay']:
    #         print(info['transcript_type'])
    #     continue
    # if 'tag' in info and 'readthrough_transcript' in info['tag']:
    #     continue
    if gene_id in repartition:
        if transcript_id in repartition[gene_id]:
            repartition[gene_id][transcript_id].add((chr, start, end))
        else:
            repartition[gene_id][transcript_id] = {(chr, start, end)}
    else:
        repartition[gene_id] = {transcript_id : {(chr, start, end)}}

In [18]:
print(len(repartition))
n_isoform_no_filter = [len(repartition[k]) for k in repartition]
sum(n_isoform_no_filter)

19347


88546

In [19]:
repartition_no_dup = dict()
for gene in repartition:
    repartition_no_dup[gene] = dict()
    for transcript in repartition[gene]:
        if repartition[gene][transcript] not in repartition_no_dup[gene].values():
            repartition_no_dup[gene][transcript] = repartition[gene][transcript]

In [20]:
print(len(repartition_no_dup))
n_isoform_no_filter = [len(repartition_no_dup[k]) for k in repartition_no_dup]
sum(n_isoform_no_filter)

19347


75508

In [21]:
coding_transcript = []
for gene in repartition_no_dup.values():
    coding_transcript.extend(list(gene.keys()))
print(len(coding_transcript))
coding_transcript = set(coding_transcript)

75508


In [22]:
def keeprow(x):
    if x[2] == 'gene':
        return infogff2dict(x[8])['gene_id'] in coding_gene
    else:
        return infogff2dict(x[8])['transcript_id'] in coding_transcript

In [23]:
row_kept = feature_human_annotation.apply(keeprow, axis=1)

In [24]:
feature_human_annotation = feature_human_annotation[row_kept]
feature_human_annotation.to_csv('gencode.hs.coding_gene.coding_transcript.no_readthrougth.v45.gff', header=None, sep='\t', index=False)

In [25]:
feature_human_annotation

,0,1,2,3,4,5,6,7,8
58,chr1,HAVANA,gene,65419,71585,.,+,.,ID=ENSG00000186092.7;gene_id=ENSG00000186092.7...
59,chr1,HAVANA,transcript,65419,71585,.,+,.,ID=ENST00000641515.2;Parent=ENSG00000186092.7;...
62,chr1,HAVANA,CDS,65565,65573,.,+,0,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
65,chr1,HAVANA,CDS,69037,70008,.,+,0,ID=CDS:ENST00000641515.2;Parent=ENST0000064151...
314,chr1,HAVANA,gene,450740,451678,.,-,.,ID=ENSG00000284733.2;gene_id=ENSG00000284733.2...
...,...,...,...,...,...,...,...,...,...
3425338,chrM,ENSEMBL,transcript,14149,14673,.,-,.,ID=ENST00000361681.2;Parent=ENSG00000198695.2;...
3425340,chrM,ENSEMBL,CDS,14149,14673,.,-,0,ID=CDS:ENST00000361681.2;Parent=ENST0000036168...
3425345,chrM,ENSEMBL,gene,14747,15887,.,+,.,ID=ENSG00000198727.2;gene_id=ENSG00000198727.2...
3425346,chrM,ENSEMBL,transcript,14747,15887,.,+,.,ID=ENST00000361789.2;Parent=ENSG00000198727.2;...
